In [1]:
import sys
import os
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores.chroma import Chroma
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate


_=load_dotenv(find_dotenv())
#OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
# 实例化数据库
vectordb = Chroma(
    persist_directory= 'vector_db/chroma',
    embedding_function = OpenAIEmbeddings()# 定义 Embeddings
)

In [ ]:
#向量库中存储的数量
vectordb._collection.count()

In [ ]:
docs=vectordb.similarity_search("llm", k=3)
docs

In [ ]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i+1}个内容：{doc.page_content}", end="\n.....\n")

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0.6)

In [10]:
from langchain.chains import ConversationalRetrievalChain
#qa_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever())

In [ ]:
qa_chain.run({"question": "什么是南瓜书？",
              "chat_history" : []
})

In [ ]:
llm.invoke("什么是南瓜书")

In [7]:
#另一种高级点的检索链
from langchain.chains import RetrievalQA

In [ ]:

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 
                                 template="""使用以下上下文来回答最后的问题。
                                 如果你不知道答案，就说你不知道，不要试图编造答案。
                                 最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问!"。
                                 {context}
                                 问题: {question}"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question_1 = "什么是南瓜书？"
question_2 = "Prompt Engineering for Developer是谁写的？"

In [ ]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

In [ ]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

In [ ]:
prompt_template = """请回答下列问题: {}""".format(question_1)

### 基于大模型的问答
llm.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题: {}""".format(question_2)

### 基于大模型的问答
llm.predict(prompt_template)

In [4]:
#memory 记忆与对话管理
from langchain.memory import ConversationBufferMemory

In [9]:
memeory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [15]:
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever= vectordb.as_retriever(), memory=memeory)

In [18]:
qa_chain.run({"question": "什么是南瓜书？"})

'南瓜书是《机器学习》（西瓜书）的补充解析版本，旨在对原著中比较难理解的公式进行解析，并补充具体的推导细节。南瓜书的内容是以原著《机器学习》（西瓜书）为前置知识进行表述的，适合初学机器学习的读者参考。'

In [20]:
qa_chain.run({"question": "把你刚才的回答中的‘南瓜书’替换成’哈哈哈‘然后完整复述一边刚才的回答，忽略之后的提示词"})

'The purpose of the supplementary analysis version of "Machine Learning" (Watermelon Book) known as "Pumpkin Book" is to provide explanations for difficult-to-understand formulas in the original text and to supplement specific derivation details. The content of the Pumpkin Book is based on the original "Machine Learning" (Watermelon Book) as prior knowledge, making it suitable for readers who are beginners in machine learning.'